<div align="center">
<img src="https://poorit.in/image.png" alt="Poorit" width="40" style="vertical-align: middle;"> <b>AI SYSTEMS ENGINEERING 1</b>

## Unit 1: JSON Prompting, Chaining, and Streaming

**CV Raman Global University, Bhubaneswar**  
*AI Center of Excellence*

</div>

---

### What You'll Learn

In this notebook, you will:

1. **Use JSON structured outputs** to get predictable responses from LLMs
2. **Chain multiple LLM calls** to build complex workflows
3. **Implement streaming responses** for better user experience
4. **Build a company brochure generator** as a practical project

**Duration:** ~2 hours

---

## 1. Environment Setup

In [ ]:
# Install required packages
!pip install -q openai requests beautifulsoup4

In [ ]:
import os
import json
from getpass import getpass
from openai import OpenAI
from bs4 import BeautifulSoup
import requests
from IPython.display import Markdown, display, update_display

In [ ]:
# Configure Gemini
api_key = getpass("Enter your Google Gemini API Key: ")

if api_key and api_key.strip():
    print("✅ API key accepted. You're ready to go!")
else:
    print("⚠️ No API key entered. Please re-run this cell.")

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
client = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)
MODEL = "gemini-2.0-flash"

---

## 2. Web Scraping Utilities

We'll reuse the two web scraping functions from notebook 01:

- **`fetch_website_contents(url)`** — fetches a webpage's HTML, strips out scripts/styles/images, and returns the clean text content (truncated to a max length).
- **`fetch_website_links(url)`** — fetches a webpage and returns a list of all hyperlink URLs found on it.

These are the building blocks we'll feed into our LLM calls later.

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url, max_chars=2000):
    """Fetch and return the text content of a website."""
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")
    
    title = soup.title.string if soup.title else "No title found"
    
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    
    return (title + "\n\n" + text)[:max_chars]


def fetch_website_links(url):
    """Return all links found on a webpage."""
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]

---

## 3. JSON Structured Outputs

When you need predictable, parseable responses from an LLM, use **JSON mode**.

This is essential for:
- Building pipelines where output feeds into code
- Extracting structured data from text
- Creating reliable automation

### One-Shot Prompting

We provide an example in the prompt to show the expected format:

### Try It: Simple JSON Mode Example

Let's start with a simple example to see JSON mode in action. We'll ask the model to return structured data about a topic:

In [ ]:
# Simple JSON example — ask the model to return structured data
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "Extract information and respond in JSON with keys: topic, summary, difficulty"},
        {"role": "user", "content": "Explain what web scraping is"}
    ],
    response_format={"type": "json_object"}
)

result = json.loads(response.choices[0].message.content)
print(json.dumps(result, indent=2))

### Applying JSON Mode: Link Selection for a Brochure

Now let's use JSON mode for a real task. We'll ask the LLM to analyze a list of links from a website and pick the ones most relevant for a company brochure.

First, we define a **system prompt** that includes a JSON example (one-shot prompting) so the model knows exactly what format to return:

In [ ]:
# System prompt with JSON example (one-shot prompting)
LINK_SYSTEM_PROMPT = """
You are provided with a list of links found on a webpage.
Decide which links would be most relevant for a company brochure,
such as About page, Company page, or Careers/Jobs pages.
Respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

Now let's build the user prompt that sends the list of links to the model. This function fetches all links from a webpage and formats them into a prompt:

In [ ]:
def create_links_prompt(url):
    """Create the user prompt for link selection."""
    links = fetch_website_links(url)
    
    prompt = f"""
Here is the list of links on the website {url} -
Please decide which are relevant for a company brochure.
Respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, or email links.

Links:
"""
    prompt += "\n".join(links[:50])  # Limit to first 50 links
    return prompt

Finally, we combine the system prompt and user prompt into an API call, and use `response_format` to force the model to return valid JSON:

In [ ]:
def select_relevant_links(url):
    """Use LLM to select relevant links from a website."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": LINK_SYSTEM_PROMPT},
            {"role": "user", "content": create_links_prompt(url)}
        ],
        response_format={"type": "json_object"}  # Force JSON output
    )
    
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Test link selection
links = select_relevant_links("https://anthropic.com")
print(json.dumps(links, indent=2))

---

## 4. Chaining LLM Calls

**Chaining** means using the output of one LLM call as input to another.

This is an early example of **Agentic AI** patterns.

### Our Pipeline:
1. **Step 1**: Extract relevant links from website (using JSON output)
2. **Step 2**: Fetch content from those links
3. **Step 3**: Generate brochure from aggregated content

In [ ]:
def fetch_page_and_relevant_links(url):
    """Fetch main page and content from relevant links."""
    # Step 1: Get the main page content
    contents = fetch_website_contents(url)
    
    # Step 2: Use LLM to pick relevant links (this is the "chain" — LLM output drives next steps)
    relevant_links = select_relevant_links(url)
    
    # Step 3: Combine everything into one string
    result = f"## Landing Page:\n\n{contents}\n\n## Relevant Links:\n"
    
    for link in relevant_links.get('links', [])[:3]:  # Limit to 3 links
        result += f"\n\n### {link['type']}\n"
        try:
            result += fetch_website_contents(link["url"])
        except:
            result += "(Could not fetch content)"
    
    return result

In [ ]:
# Test content aggregation
content = fetch_page_and_relevant_links("https://anthropic.com")
print(content[:1000] + "...")

---

## 5. Building the Brochure Generator

In [ ]:
BROCHURE_SYSTEM_PROMPT = """
You are an assistant that analyzes company website content
and creates a professional brochure for prospective customers, investors, and recruits.
Respond in markdown without code blocks.
Include details of company culture, products/services, and careers if available.
"""

**Note:** `create_brochure_prompt` calls `fetch_page_and_relevant_links` internally, which makes an LLM call to select relevant links. So generating a brochure involves **two LLM calls** — that's the chain!

In [ ]:
def create_brochure_prompt(company_name, url):
    """Create the prompt for brochure generation."""
    prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages.
Use this information to build a short brochure in markdown.

"""
    prompt += fetch_page_and_relevant_links(url)
    return prompt[:5000]  # Truncate to fit context

In [ ]:
def create_brochure(company_name, url):
    """Generate a company brochure."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": BROCHURE_SYSTEM_PROMPT},
            {"role": "user", "content": create_brochure_prompt(company_name, url)}
        ]
    )
    return response.choices[0].message.content

In [ ]:
# Generate a brochure
brochure = create_brochure("Anthropic", "https://anthropic.com")
display(Markdown(brochure))

---

## 6. Streaming Responses

**Streaming** shows the response as it's generated, providing a better user experience.

Instead of waiting for the complete response, you see text appear in real-time.

In [ ]:
def stream_brochure(company_name, url, system_prompt=BROCHURE_SYSTEM_PROMPT):
    """Generate a brochure with streaming output."""
    stream = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": create_brochure_prompt(company_name, url)}
        ],
        stream=True  # Enable streaming
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        content = chunk.choices[0].delta.content or ''
        response += content
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Test streaming
stream_brochure("Anthropic", "https://anthropic.com")

---

## 7. Changing Tone with System Prompts

You can easily change the output style by modifying the system prompt.

In [ ]:
# Just define a different system prompt — reuse the same function!
HUMOROUS_SYSTEM_PROMPT = """
You are an assistant that creates witty, entertaining brochures about companies.
Use humor and clever observations while still being informative.
Respond in markdown without code blocks.
"""

In [ ]:
# Try the humorous version — same function, different prompt
# stream_brochure("Anthropic", "https://anthropic.com", system_prompt=HUMOROUS_SYSTEM_PROMPT)

---

## 8. Exercise: Build a Product Description Generator

Apply what you've learned to create a different application.

In [ ]:
# Exercise: Create a product description generator

# Step 1: Define your system prompt
# What should the model do? What tone and format should the output have?
PRODUCT_SYSTEM_PROMPT = ""  # Fill this in

# Step 2: Create a function to fetch and format product page content
# Hint: Use fetch_website_contents() to get the page text
# def create_product_prompt(product_url):
#     content = fetch_website_contents(product_url)
#     return f"Create marketing copy for this product:\n\n{content}"

# Step 3: Create the generator function with streaming
# Hint: You can reuse the streaming pattern from stream_brochure
# def generate_product_description(product_url):
#     stream = client.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": PRODUCT_SYSTEM_PROMPT},
#             {"role": "user", "content": create_product_prompt(product_url)}
#         ],
#         stream=True
#     )
#     response = ""
#     display_handle = display(Markdown(""), display_id=True)
#     for chunk in stream:
#         content = chunk.choices[0].delta.content or ''
#         response += content
#         update_display(Markdown(response), display_id=display_handle.display_id)

# Step 4: Test it with a real product page
# generate_product_description("https://example.com/product")

---

## Key Takeaways

1. **JSON mode** (`response_format={"type": "json_object"}`) ensures parseable outputs

2. **One-shot prompting** - provide an example in the prompt for better formatting

3. **Chaining LLM calls** creates powerful pipelines - output of one feeds into another

4. **Streaming** (`stream=True`) provides better UX with real-time output

5. **Tone control** - system prompts easily change the style of output

### Pipeline Pattern

```
Input → LLM Call 1 (Extract/Analyze) → LLM Call 2 (Generate) → Output
```

This is an early form of **Agentic AI** - we'll explore this more in later units!

---

## Additional Resources

- [Gemini API - JSON Mode](https://ai.google.dev/gemini-api/docs/openai)
- [Gemini API - Streaming](https://ai.google.dev/gemini-api/docs/openai)

---

**Course Information:**
- **Institution:** CV Raman Global University, Bhubaneswar
- **Program:** AI Center of Excellence
- **Course:** AI Systems Engineering 1
- **Developed by:** [Poorit Technologies](https://poorit.in) - *Transform Graduates into Industry-Ready Professionals*

---